In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,Dropout,Bidirectional
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [3]:
PATH ='C:/Users/ckdrj/Google 드라이브/baseball/data/total/'
#PATH = 'C:/Users/LIM CHANG GEON/Google 드라이브/baseball/data/total'
hiter = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv")
pitcher = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv")
pitcher["ERA"] = 27*pitcher.ER/pitcher.INN2
hiter['AVG']= hiter['HIT']/hiter['AB']


In [2]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back), 0])
        dataY.append(signal_data[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [4]:
def univariate_ans(data, look_back,team,feature):
    signal_data = data[data.T_ID==team][[feature]]
    # 데이터 전처리
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    signal_data = scaler.fit_transform(signal_data)
    last_data = signal_data[-10:]
    # 데이터셋 생성
    x_train, y_train = create_dataset(signal_data, look_back)
    
    x_train = np.reshape(x_train,(x_train.shape[0],look_back,1)) #(size, timestamp,feature)
    #x_test = np.reshape(x_test,(x_test.shape[0],look_back,1))
    
    return x_train, y_train,scaler, look_back, last_data

In [6]:
def run_stateful_ans(x_train,  y_train, look_back,box):
    model = Sequential()
    model.add(LSTM(box, batch_input_shape=(1, look_back, 1), stateful=True))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

    model.fit(x_train, y_train, epochs=100, batch_size=1, verbose=1, callbacks=[early_stop])
    return model

In [10]:
x_train, y_train,scaler, look_back,last_data = univariate_ans(pitcher,10,'SS',"ERA")
model = run_stateful_ans(x_train,  y_train, look_back,10)
pred_date = 20 
seq_in = [i[0] for i in last_data]
seq_out = []
for i in range(pred_date):
    #sample_in = np.array(seq_in)
    sample_in = np.reshape(seq_in, (1, look_back,1)) # batch_size, feature
    pred_out = model.predict(sample_in)[0][0]
    seq_out.append(pred_out)
    seq_in.append(pred_out)
    seq_in.pop(0)
ans = scaler.inverse_transform(np.array(seq_out).reshape(pred_date,1))
ans_mean=ans.mean()
print(ans_mean)

Epoch 1/100
134/134 [==============================] - 1s 11ms/step - loss: 0.1036
Epoch 2/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0623
Epoch 3/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0681
Epoch 4/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0621
Epoch 5/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0635
Epoch 6/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0638
Epoch 7/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0619
Epoch 8/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0622
Epoch 9/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0582
Epoch 10/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0598
Epoch 11/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0632
Epoch 12/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0590
Epoch 00012:

In [9]:
x_train, y_train,scaler, look_back,last_data = univariate_ans(hiter,10,'SS',"AVG")
model = run_stateful_ans(x_train,  y_train, look_back,10)
pred_date = 20 
seq_in = [i[0] for i in last_data]
seq_out = []
for i in range(pred_date):
    #sample_in = np.array(seq_in)
    sample_in = np.reshape(sample_in, (1, look_back,1)) # batch_size, feature
    pred_out = model.predict(sample_in)[0][0]
    seq_out.append(pred_out)
    seq_in.append(pred_out)
    seq_in.pop(0)
ans = scaler.inverse_transform(np.array(seq_out).reshape(pred_date,1))
ans_mean=ans.mean()
print(ans_mean)

Epoch 1/100
134/134 [==============================] - 1s 10ms/step - loss: 0.1570
Epoch 2/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0476
Epoch 3/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0527
Epoch 4/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0430
Epoch 5/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0495
Epoch 6/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0469
Epoch 7/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0419
Epoch 8/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0427
Epoch 9/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0461
Epoch 10/100
134/134 [==============================] - 1s 5ms/step - loss: 0.0458
Epoch 00010: early stopping
0.26409534
